# Data Preprocessing 

This notebook includes techniques for data preprocessing regarding: 

1. Imputation of missing values

    * mean, mode imputation
    * using k-Nearest Neighbors 


2. Encoding & Feature Engineering
    * one-Hot encode
    * integer encode
    * binary encode
    * bin-counting scheme


3. Standardization & Normalization 


---------------------

Sample data: UCI diabetes readmission 

https://archive.ics.uci.edu/ml/datasets/Diabetes+130-US+hospitals+for+years+1999-2008#

https://www.hindawi.com/journals/bmri/2014/781670/tab1/


In [2]:
# load packages

import pandas as pd 
import numpy as np 

In [3]:
# set up display options for dataframe 

pd.options.mode.chained_assignment = None
pd.set_option('display.max_columns', 999)
# pd.options.display.max_columns = 999

In [8]:
# load data 
data = pd.read_csv('Desktop/code/cisc6930_dm/data/dataset_diabetes/diabetic_data.csv')
print(data.shape)

# data types and descriptive stats
# print(data.info())
print(data.describe())

# make a copy of the dataframe for preprocessing
# create an empty dataframe to store data later 
df_clean = pd.DataFrame()
df = data.copy(deep=True)

(101766, 50)
       encounter_id   patient_nbr  admission_type_id  \
count  1.017660e+05  1.017660e+05      101766.000000   
mean   1.652016e+08  5.433040e+07           2.024006   
std    1.026403e+08  3.869636e+07           1.445403   
min    1.252200e+04  1.350000e+02           1.000000   
25%    8.496119e+07  2.341322e+07           1.000000   
50%    1.523890e+08  4.550514e+07           1.000000   
75%    2.302709e+08  8.754595e+07           3.000000   
max    4.438672e+08  1.895026e+08           8.000000   

       discharge_disposition_id  admission_source_id  time_in_hospital  \
count             101766.000000        101766.000000     101766.000000   
mean                   3.715642             5.754437          4.395987   
std                    5.280166             4.064081          2.985108   
min                    1.000000             1.000000          1.000000   
25%                    1.000000             1.000000          2.000000   
50%                    1.000000       

In [97]:
# check unique id 

len(data.patient_nbr.unique())
len(data.encounter_id.unique())

# df = data.groupby("patient_nbr").filter(lambda x: len(x) > 1)
# df.head()

# df_115196778 = data[data['patient_nbr'] == 115196778]
# df_115196778.to_csv("Desktop/instance.csv", index=True, header=True)

101766

## 1. Check and imput missing values 

count the number and percentage of missing values in each variables


In [98]:
data.describe() 

,encounter_id,patient_nbr,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,num_lab_procedures,num_procedures,num_medications,number_outpatient,number_emergency,number_inpatient,number_diagnoses
count,1.017660e+05,1.017660e+05,101766.000000,101766.000000,101766.000000,101766.000000,101766.000000,101766.000000,101766.000000,101766.000000,101766.000000,101766.000000,101766.000000
mean,1.652016e+08,5.433040e+07,2.024006,3.715642,5.754437,4.395987,43.095641,1.339730,16.021844,0.369357,0.197836,0.635566,7.422607
std,1.026403e+08,3.869636e+07,1.445403,5.280166,4.064081,2.985108,19.674362,1.705807,8.127566,1.267265,0.930472,1.262863,1.933600
min,1.252200e+04,1.350000e+02,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,1.000000
25%,8.496119e+07,2.341322e+07,1.000000,1.000000,1.000000,2.000000,31.000000,0.000000,10.000000,0.000000,0.000000,0.000000,6.000000
50%,1.523890e+08,4.550514e+07,1.000000,1.000000,7.000000,4.000000,44.000000,1.000000,15.000000,0.000000,0.000000,0.000000,8.000000
75%,2.302709e+08,8.754595e+07,3.000000,4.000000,7.000000,6.000000,57.000000,2.000000,20.000000,0.000000,0.000000,1.000000,9.000000
max,4.438672e+08,1.895026e+08,8.000000,28.000000,25.000000,14.000000,132.000000,6.000000,81.000000,42.000000,76.000000,21.000000,16.000000


In [99]:
# the count and percentage of missing values 

# df.apply(pd.value_counts())
count = [data[c].value_counts() for c in list(data.select_dtypes(include=['O']).columns)]
# find unique values and missings 
# data.readmitted.unique()

In [100]:
# replace all the ? with nan to count with isnull function 

df = data.replace('?', np.nan) # or .replace('-', {0: None})
# df.head()

In [101]:
# a function that output a dataframe to preview the missing values 
# and the % of missing values in each column

def missing_values_table(df):
        mis_val = df.isnull().sum()
        mis_val_percent = 100 * df.isnull().sum() / len(df)
        mis_val_table = pd.concat([mis_val, mis_val_percent], axis=1)
        
        mis_val_table_ren_columns = mis_val_table.rename(
        columns = {0 : 'Missing Values', 1 : '% of Total Values'})
        
        # keep only the rows that missing value != 0
        # sort the rows by % of missing values 
        mis_val_table_ren_columns = mis_val_table_ren_columns[
            mis_val_table_ren_columns.iloc[:,1] != 0].sort_values(
        '% of Total Values', ascending=False).round(1)
        
        print ("Your selected dataframe has " + str(df.shape[1]) + " columns.\n"      
            "There are " + str(mis_val_table_ren_columns.shape[0]) +
              " columns that have missing values.")
        
        return mis_val_table_ren_columns

missing_values_table(df)

Your selected dataframe has 50 columns.
There are 7 columns that have missing values.


,Missing Values,% of Total Values
weight,98569,96.9
medical_specialty,49949,49.1
payer_code,40256,39.6
race,2273,2.2
diag_3,1423,1.4
diag_2,358,0.4
diag_1,21,0.0


In [4]:
# drop columns with large % of missing values 

data = data.drop(['weight','payer_code','medical_specialty'], axis = 1) 

# cols = ['weight', 'payer_code', 'medical_specialty']
# data.drop(cols, axis = 1, inplace = True)

data.head()

,encounter_id,patient_nbr,race,gender,age,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,num_lab_procedures,num_procedures,num_medications,number_outpatient,number_emergency,number_inpatient,diag_1,diag_2,diag_3,number_diagnoses,max_glu_serum,A1Cresult,metformin,repaglinide,nateglinide,chlorpropamide,glimepiride,acetohexamide,glipizide,glyburide,tolbutamide,pioglitazone,rosiglitazone,acarbose,miglitol,troglitazone,tolazamide,examide,citoglipton,insulin,glyburide-metformin,glipizide-metformin,glimepiride-pioglitazone,metformin-rosiglitazone,metformin-pioglitazone,change,diabetesMed,readmitted
0,2278392,8222157,Caucasian,Female,[0-10),6,25,1,1,41,0,1,0,0,0,250.83,?,?,1,None,None,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,NO
1,149190,55629189,Caucasian,Female,[10-20),1,1,7,3,59,0,18,0,0,0,276,250.01,255,9,None,None,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,Up,No,No,No,No,No,Ch,Yes,>30
2,64410,86047875,AfricanAmerican,Female,[20-30),1,1,7,2,11,5,13,2,0,1,648,250,V27,6,None,None,No,No,No,No,No,No,Steady,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,Yes,NO
3,500364,82442376,Caucasian,Male,[30-40),1,1,7,2,44,1,16,0,0,0,8,250.43,403,7,None,None,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,Up,No,No,No,No,No,Ch,Yes,NO
4,16680,42519267,Caucasian,Male,[40-50),1,1,7,1,51,0,8,0,0,0,197,157,250,5,None,None,No,No,No,No,No,No,Steady,No,No,No,No,No,No,No,No,No,No,Steady,No,No,No,No,No,Ch,Yes,NO


### Mode Imputation

For categorical data which are not ordinal 

In [5]:
# for race: mode imputation 

# get the number of the largest instance 
max_race_count = max(data.race.value_counts()) 

# access 'Caucasian' 
max_race = data.race.value_counts()[data.race.value_counts() == max_race_count].index[0] 

# imput missing values with the largest number of race 'Caucasian' 
data.race.replace('?', max_race, inplace = True)
data.race.unique()

array(['Caucasian', 'AfricanAmerican', 'Other', 'Asian', 'Hispanic'],
      dtype=object)

### Scikit-Learn - Imputer


In [21]:
from numpy import nan
X = np.array([[ nan, 0,   3  ],
              [ 3,   7,   9  ],
              [ 3,   5,   2  ],
              [ 4,   nan, 6  ],
              [ 8,   8,   1  ]])
y = np.array([14, 16, -1,  8, -5])

In [22]:
from sklearn.preprocessing import Imputer
imp = Imputer(strategy='mean')
X2 = imp.fit_transform(X)
X2

/anaconda3/lib/python3.6/site-packages/sklearn/utils/deprecation.py:58: DeprecationWarning: Class Imputer is deprecated; Imputer was deprecated in version 0.20 and will be removed in 0.22. Import impute.SimpleImputer from sklearn instead.
  warnings.warn(msg, category=DeprecationWarning)


array([[4.5, 0. , 3. ],
       [3. , 7. , 9. ],
       [3. , 5. , 2. ],
       [4. , 5. , 6. ],
       [8. , 8. , 1. ]])

## 2. Encoding & Feature Engineering

The type of data in this dataset includes: 

**Continuous**
3. time_in_hospital	
4. num_lab_procedures
5. num_medications
6. number_outpatient
7. number_emergency
8. number_inpatient
12. number_diagnoses

Discrete: 
1. encounter_id
2. patient_nbr
9. diag_1: with V51?? 
10. diag_2: missing value, V15, E813
11. diag_3: missing value, V, E

Categorical: 

a) Ordinal categorical: 
3. age: range 
4. weight: range with missing 

b) binary: 
31. change: 'No', 'Ch'
32. diabetesMed: 'No', 'Yes'

c) with one value
23. examide: 'No'
24. citoglipton: 'No'

d) one-hot encode 
1. race: 5 with missing 
2. gender: unknown 

6. medical_specialty: categorical 
7. max_glu_serum: 'None', '>300', 'Norm', '>200'
8. A1Cresult: 'None', '>7', '>8', 'Norm']



In [104]:
# buiding an dataframe to store clean data 

df_clean['encounter_id'] = data.encounter_id
df_clean['patient_id'] = data.patient_nbr

* For continuous values: we keep them as it it

In [105]:
# number_outpatient: continuous value 
# number_emergency: continuous value 
# number_inpatient: continuous value 

# data.num_lab_procedures.unique()

df_clean['number_outpatient'] = data['number_outpatient']
df_clean['number_emergency'] = data['number_emergency']
df_clean['number_inpatient'] = data['number_inpatient']

### 1) Binary Encoding

In [106]:
# Check class label: code into binary for binary classification problem 
# 0: 'NO' and '>30'
# 1: '<30'

data['readmitted'] = [0 if x in ('No', '>30') else 1 for x in data['readmitted']]
df_clean['readmitted'] = data['readmitted']

# data.readmitted[df.readmitted == '<30'] = 1
# data.readmitted[df.readmitted == 'NO'] = 0
# data.readmitted[df.readmitted == '>30'] = 0


# data = data.replace({'readmitted':'NO'}, 0)
# data = data.replace({'readmitted':'>30'}, 0)
# data = data.replace({'readmitted':'<30'}, 1)

# data.head()
# data.shape

In [107]:
# for gender: have 3 unknowns

# gender_code = {'Female':0,'Male':1,'Unknown/Invalid':1} # build a dictionary for encoding 
# df.gender = [gender_code[x] for x in df.gender] # 

df_clean['is_male'] = [0 if x == 'female' else 0 for x in data.gender]
# df_clean.head()

In [108]:
# Change of medications: binary encoding 

change_code = {'No':0,'Ch':1}
data.change = [change_code[x] for x in data.change]
df_clean['change_in_medications'] = data.change

In [109]:
# Diabetes medications

diabetesMed_code = {'No':0,'Yes':1}
data.diabetesMed = [diabetesMed_code[x] for x in data.diabetesMed]
df_clean['diabetesMed'] = data.diabetesMed

### 2) One-hot Encoding

In [110]:
# for race: One-hot encoding 

df_clean['race_white'] = [1 if x == 'Caucasian' else 0 for x in data.race]
df_clean['race_aa'] = [1 if x == 'AfricanAmerican' else 0 for x in data.race]
df_clean['race_hispanic'] = [1 if x == 'Hispanic' else 0 for x in data.race]
df_clean['race_asian'] = [1 if x == 'Asian' else 0 for x in data.race]


In [23]:
# # another method: DictVectorizer
# sklearn.preprocessing.OneHotEncoder
# sklearn.feature_extraction.FeatureHasher
# https://jakevdp.github.io/PythonDataScienceHandbook/05.04-feature-engineering.html
# race = data.race

# from sklearn.feature_extraction import DictVectorizer
# vec = DictVectorizer(sparse=False, dtype=int)
# vec_race = vec.fit_transform(race)

* For categorical data without order but have large number of categories, it doesn't make since to do integer encoding, since some algorithms will take the integer values as weight. 

* The solution is to collapse categories into a few and then do OHE for each 

In [111]:
# admission type id: OHE
# coded 7 categories into 5 columns based on their meanings 
# 5: not avilable, 6: NULL, 8: Not Mapped 

# data.admission_type_id.unique()

df_clean['admission_emergency_urgent'] = [1 if x in (1,2) else 0 for x in data.admission_type_id]
df_clean['admission_elective'] = [1 if x==3 else 0 for x in data.admission_type_id]
df_clean['admission_newborn'] = [1 if x==4 else 0 for x in data.admission_type_id]
df_clean['admission_trauma'] = [1 if x==7 else 0 for x in data.admission_type_id]
df_clean['admission_unknown'] = [1 if x in (5,6,8) else 0 for x in data.admission_type_id]

# df_clean.head()

In [112]:
# discharge_disposition_id: OHE after collapse

# df.discharge_disposition_id.unique()

# Collapsed Multiple discharge_disposition_ids into fewer categories:<br>
# discharge_disposition_unknown: 18, 25, 26<br>
# discharge_disposition_expired: 11, 19, 20, 21<br>
# discharge_disposition_home_other_facility: 1, 2, 3, 4, 5, 6, 7, 8, 10, 13, 14, 16, 22, 23, 24, 27, 28, 29, 30<br>
# discharge_disposition_current_hospital_patient: 9, 12, 15, 17
    
df_clean['discharge_diposition_unknown'] = [1 if x in (18, 25, 26) else 0 for x in data.discharge_disposition_id]
df_clean['discharge_diposition_expired'] = [1 if x in (11, 19, 20, 21) else 0 for x in data.discharge_disposition_id]
df_clean['discharge_diposition_home_other_facility'] = [1 if x in (1, 2, 3, 4, 5, 6, 7, 8, 10, 13, 14, 16, 22, 23, 24, 27, 28, 29, 30) else 0 for x in data.discharge_disposition_id]
df_clean['discharge_diposition_current_hospital_patient'] = [1 if x in (9, 12, 15, 17) else 0 for x in data.discharge_disposition_id]


In [113]:
# admission_source_id: OHE after collapse
# encoded into 3+1 categories: emergency, referral, transfer

# df.admission_source_id.unique()

df_clean['admission_source_emergency_room'] = [1 if x==7 else 0 for x in data.admission_source_id]
df_clean['admission_source_physician_referral'] = [1 if x==1 else 0 for x in data.admission_source_id]
df_clean['admission_source_transfer_hospital_health_care_facility_clinic'] = [1 if x in (4,6,2) else 0 for x in data.admission_source_id]


### 3) Bin-counting Scheme


In [114]:
# for age: Bin-counting scheme
# orginally was categorical range but have order 
# so we code them into 0, 1, 2 

youth = ['[0-10)', '[10-20)']
adult = ['[20-30)', '[30-40)', '[40-50)']
elderly = ['[50-60)','[60-70)','[70-80)','[80-90)','[90-100)']

age_codes = []
for x in data.age:
    if x in youth:
        age_codes.append(0)
    elif x in adult:
        age_codes.append(1)
    else:
        age_codes.append(2)
        
data.age = age_codes 
df_clean['age'] = data.age
# df_clean['age'] = data['age']

# df_clean.head()
# df_clean['age_youth'] = [1 if x==0 else 0 for x in data.age]
# df_clean['age_adult'] = [1 if x==1 else 0 for x in data.age]
# df_clean['age_elderly'] = [1 if x==2 else 0 for x in data.age]

### 4) Integer Encoding

In [115]:
# max_glu_serum: integer encoding 

max_glu_serum_code = {'None':0,'Norm':1,'>200':2,'>300':3}
data.max_glu_serum = [max_glu_serum_code[x] for x in data.max_glu_serum]
df_clean['max_glu_serum'] = data.max_glu_serum

# data.max_glu_serum.unique()

In [116]:
# A1Cresult: integer encoding 

A1Cresult_code = {'None':0,'Norm':1,'>7':2,'>8':3}
data.A1Cresult = [A1Cresult_code[x] for x in data.A1Cresult]
df_clean['A1Cresult'] = data.A1Cresult


Integer encoding for continuous values:

We'd like to group some continuous variables into ranges and code into integer to better capture the differences

In [117]:
# time_in_hospital: code into range and then integer encoding 
# data.time_in_hospital.unique()

for i, row in data.time_in_hospital.iteritems():
    if row <= 4:
        data.at[i, 'time_in_hospital'] = 0
    if (row > 4) & (row <= 9):
        data.at[i, 'time_in_hospital'] = 1
    if (row >9):
        data.at[i, 'time_in_hospital'] = 2

df_clean['time_in_hospital'] = data['time_in_hospital']



In [118]:
# num_lab_procedures: code into range and then integer encoding 

# df.num_lab_procedures.unique()

for i, row in data.num_lab_procedures.iteritems():
    if row <= 30:
        data.at[i, 'num_lab_procedures'] = 0
    if (row > 30) & (row <=60):
        data.at[i, 'num_lab_procedures'] = 1
    if (row > 60):
        data.at[i, 'num_lab_procedures'] = 2
df_clean['num_lab_procedures'] = data['num_lab_procedures']


In [119]:
# num_procedures: code into range and then integer encoding 

for i, row in data.num_procedures.iteritems():
    if row <= 2:
        data.at[i, 'num_procedures'] = 0
    if (row > 2) & (row <= 4):
        data.at[i, 'num_procedures'] = 1
    if (row > 4):
        data.at[i, 'num_procedures'] = 2
        
df_clean['num_procedures'] = data['num_procedures']

In [120]:
# num_medications: code into range and then integer encoding 
# df.num_medications.unique()

for i, row in data.num_medications.iteritems():
    if row <= 20:
        data.at[i, 'num_medications'] = 0
    if (row > 20) & (row <= 30):
        data.at[i, 'num_medications'] = 1
    if (row > 30):
        data.at[i, 'num_medications'] = 2
df_clean['num_medications'] = data['num_medications']

In [121]:
# number_diagnoses: code into range and then integer encoding 
# data.number_diagnoses.unique()

for i, row in data.number_diagnoses.iteritems():
    if row <= 4:
        data.at[i, 'number_diagnoses'] = 0
    if (row > 4) & (row <= 6):
        data.at[i, 'number_diagnoses'] = 1
    if (row > 6) & (row <= 8):
        data.at[i, 'number_diagnoses'] = 2
    if (row > 8):
        data.at[i, 'number_diagnoses'] = 3
        
df_clean['number_diagnoses'] = data['number_diagnoses']


### 5) Feature Engineering 

In [122]:
# 24 features/columns of medications
medicine_col_names = data.columns[23:46]
medicine_df = data[medicine_col_names]

# create two new columns of num_medicine_change and num_total_medicine
num_medicine_change_series = medicine_df.isin(['Up', 'Down']).sum(1)
num_total_medicine_series = medicine_df.isin(['Up', 'Down', 'Steady']).sum(1)

data.insert(data.columns.get_loc('metformin-pioglitazone') + 1, 'num_total_medicine', num_total_medicine_series, allow_duplicates = True)
data.insert(data.columns.get_loc('num_total_medicine') + 1, 'num_medicine_change', num_medicine_change_series, allow_duplicates = True)


In [ ]:
# look into the distribution of each features 
# a dataframe with % of values under each feature 


In [123]:
# Encoding the Medicines Columns 
# only considering columns that have considerable amount of important data
# 14 medicine columns for now

metformin_code = {'No':0,'Steady':1,'Up':2,'Down':3}

data.metformin = [metformin_code[x] for x in data.metformin]
df_clean['metformin'] = data.metformin

data.repaglinide = [metformin_code[x] for x in data.repaglinide]
df_clean['repaglinide'] = data.repaglinide

data.nateglinide = [metformin_code[x] for x in data.nateglinide]
df_clean['nateglinide'] = data.nateglinide

data.chlorpropamide = [metformin_code[x] for x in data.chlorpropamide]
df_clean['chlorpropamide'] = data.chlorpropamide

data.glimepiride = [metformin_code[x] for x in data.glimepiride]
df_clean['glimepiride'] = data.glimepiride

data.glipizide = [metformin_code[x] for x in data.glipizide]
df_clean['glipizide'] = data.glipizide

data.glyburide = [metformin_code[x] for x in data.glyburide]
df_clean['glyburide'] = data.glyburide

data.pioglitazone = [metformin_code[x] for x in data.pioglitazone]
df_clean['pioglitazone'] = data.pioglitazone

data.rosiglitazone = [metformin_code[x] for x in data.rosiglitazone]
df_clean['rosiglitazone'] = data.rosiglitazone

data.acarbose = [metformin_code[x] for x in data.acarbose]
df_clean['acarbose'] = data.acarbose 

data.miglitol = [metformin_code[x] for x in data.miglitol]
df_clean['miglitol'] = data.miglitol

data.tolazamide = [metformin_code[x] for x in data.tolazamide]
df_clean['tolazamide'] = data.tolazamide

data.insulin = [metformin_code[x] for x in data.insulin]
df_clean['insulin'] = data.insulin

data['glyburide-metformin'] = [metformin_code[x] for x in data['glyburide-metformin']]
df_clean['glyburide_metformin'] = data['glyburide-metformin']


In [124]:
# adding two new columns to clean dataframe
# num_medicine_change and num_total_medicine

df_clean['num_medicine_change'] = data['num_medicine_change']
df_clean['num_total_medicine'] = data['num_total_medicine']


**A special feature in this data is the ICD-9 Code**


"""**ICD-9 Code Categories: https://icd.codes/icd9cm**

1) 001-139 : Infectious And Parasitic Diseases **= Infection**<br>
2) 140-239 : Neoplasms ** = Neoplasms**<br>
3) 240-279 : Endocrine, Nutritional And Metabolic Diseases, And Immunity Disorders ** = Endocrine**<br>
4) 280-289 : Diseases Of The Blood And Blood-Forming Organs ** = Blood**<br>
5) 290-319 : Mental Disorders ** = Mental**<br>
6) 320-389 : Diseases Of The Nervous System And Sense Organs ** = Nervous**<br>
7) 390-459 : Diseases Of The Circulatory System ** = Circulatory**<br>
8) 460-519 : Diseases Of The Respiratory System ** = Respiratory**<br>
9) 520-579 : Diseases Of The Digestive System ** = Digestive**<br>
10) 580-629 : Diseases Of The Genitourinary System ** = Genitourinary**<br>
11) 630-679 : Complications Of Pregnancy, Childbirth, And The Puerperium ** = Pregnancy_Childbirth**<br>
12) 680-709 : Diseases Of The Skin And Subcutaneous Tissue ** = Skin**<br>
13) 710-739 : Diseases Of The Musculoskeletal System And Connective Tissue ** = Musculoskeletal**<br>
14) 740-759 : Congenital Anomalies ** = Congenital**<br>
15) 760-779 : Certain Conditions Originating In The Perinatal Period ** = Perinatal_Condition**<br>
16) 780-799 : Symptoms, Signs, And Ill-Defined Conditions ** = Symptoms**<br>
17) 800-999 : Injury And Poisoning ** = Injury_Poisoning**<br>
18) V01-V91 : Supplementary Classification Of Factors Influencing Health Status And Contact With Health Services ** = Additional_Factors**<br>
19) E000-E999 : Supplementary Classification Of External Causes Of Injury And Poisoning ** = External_Cause**
"""

In [ ]:
# diag_1, diag_2, diag_3

col = data['diag_1']

# set the idx as diag_1+1
data.insert(data.columns.get_loc('diag_1') + 1, 'numeric_diag_1', col, allow_duplicates=True)

# type(data['numeric_diag_1'])
# data.reset_index(level = ['numeric_diag_1'])
                          
# temporarily inputting -1 in numeric_diag columns for codes with 'V' or 'E'
data.loc[data['numeric_diag_1'].str.contains("V|E")==True, ['numeric_diag_1']] = '-1'

# replace '?' with NaN in numeric_diag columns
data = data.replace({'numeric_diag_1': '?'}, np.nan)

#converting numeric_diag columns to numeric datatype
data['numeric_diag_1'] = pd.to_numeric(data.numeric_diag_1)

# creating duplicate numeric_diag_1 column as category_diag_1
col = data['numeric_diag_1']
data.insert(data.columns.get_loc('numeric_diag_1') + 1, 'category_diag_1', col, allow_duplicates = True)


In [136]:
# Encoding the categories for value range in ICD-9 Diagnosis Codes

def encode_diagnosis_categories(diag_col_name, data):
    temp_data = data
    temp_data[diag_col_name] = np.where(temp_data[diag_col_name].between(0, 140, inclusive=False), 
                                    1001, data[diag_col_name]) #'Infection', data[diag_col_name])
    temp_data[diag_col_name] = np.where(temp_data[diag_col_name].between(139, 240, inclusive=False), 
                                    1002, data[diag_col_name]) #'Neoplasms', data[diag_col_name])
    temp_data[diag_col_name] = np.where(temp_data[diag_col_name].between(239, 280, inclusive=False), 
                                    1003, data[diag_col_name]) #'Endocrine', data[diag_col_name])
    temp_data[diag_col_name] = np.where(temp_data[diag_col_name].between(279, 290, inclusive=False), 
                                    1004, data[diag_col_name]) #'Blood', data[diag_col_name])
    temp_data[diag_col_name] = np.where(temp_data[diag_col_name].between(289, 320, inclusive=False), 
                                    1005, data[diag_col_name]) #'Mental', data[diag_col_name])
    temp_data[diag_col_name] = np.where(temp_data[diag_col_name].between(319, 390, inclusive=False), 
                                    1006, data[diag_col_name]) #'Nervous', data[diag_col_name])
    temp_data[diag_col_name] = np.where(temp_data[diag_col_name].between(389, 460, inclusive=False), 
                                    1007, data[diag_col_name]) #'Circulatory', data[diag_col_name])
    temp_data[diag_col_name] = np.where(temp_data[diag_col_name].between(459, 520, inclusive=False), 
                                    1008, data[diag_col_name]) #'Respiratory', data[diag_col_name])
    temp_data[diag_col_name] = np.where(temp_data[diag_col_name].between(519, 580, inclusive=False), 
                                    1009, data[diag_col_name]) #'Digestive', data[diag_col_name])
    temp_data[diag_col_name] = np.where(temp_data[diag_col_name].between(579, 630, inclusive=False), 
                                    1010, data[diag_col_name]) #'Genitourinary', data[diag_col_name])
    temp_data[diag_col_name] = np.where(temp_data[diag_col_name].between(629, 680, inclusive=False), 
                                    1011, data[diag_col_name]) #'Pregnancy_Chidlbirth', data[diag_col_name])
    temp_data[diag_col_name] = np.where(temp_data[diag_col_name].between(679, 710, inclusive=False), 
                                    1012, data[diag_col_name]) #'Skin', data[diag_col_name])
    temp_data[diag_col_name] = np.where(temp_data[diag_col_name].between(709, 740, inclusive=False), 
                                    1013, data[diag_col_name]) #'Musculoskeletal', data[diag_col_name])
    temp_data[diag_col_name] = np.where(temp_data[diag_col_name].between(739, 760, inclusive=False), 
                                    1014, data[diag_col_name]) #'Congenital', data[diag_col_name])
    temp_data[diag_col_name] = np.where(temp_data[diag_col_name].between(759, 780, inclusive=False), 
                                    1015, data[diag_col_name]) #'Perinatal_Condition', data[diag_col_name])
    temp_data[diag_col_name] = np.where(temp_data[diag_col_name].between(779, 800, inclusive=False), 
                                    1016, data[diag_col_name]) #'Symptoms', data[diag_col_name])
    temp_data[diag_col_name] = np.where(temp_data[diag_col_name].between(799, 1000, inclusive=False), 
                                    1017, data[diag_col_name]) #'Injury_Poisoning', data[diag_col_name])
    temp_data[diag_col_name] = np.where(temp_data[diag_col_name] == -1, 
                                    1018, data[diag_col_name]) #'Additional_Factors/External_Cause', data[diag_col_name])
    return temp_data


In [137]:
cols = ['category_diag_1']
data = encode_diagnosis_categories(cols[0], data)
num_vals = [1001, 1002, 1003, 1004, 1005, 1006, 1007, 1008, 1009, 1010, 
            1011, 1012, 1013, 1014, 1015, 1016, 1017, 1018]
string_vals = ['Infection', 'Neoplasms', 'Endocrine', 'Blood', 'Mental', 'Nervous', 'Circulatory', 'Respiratory', 'Digestive', 'Genitourinary',
              'Pregnancy_Childbirth', 'Skin', 'Musculoskeletal', 'Congenital', 'Perinatal_Condition', 'Symptoms', 'Injury_Poisoning', 'Additional_Factors/External_Cause']

for i in range(len(num_vals)):
    # data.replace({col: 1001}, 'Infection')
    data = data.replace({cols[0]: num_vals[i]}, string_vals[i]) 

data = data.replace({cols[0]: np.nan}, "Unknown")

# Collapsing Multiple diag_1 categories into main categories
diag_categories = ['Circulatory', 'Endocrine', 'Respiratory', 'Digestive', 
                   'Symptoms', 'Injury_Poisoning', 'Genitourinary', 'Musculoskeletal', 'Neoplasms', 'Other']

def one_hot_encode(string_value):
    return([1 if x==string_value else 0 for x in data.category_diag_1])

df_clean['diag1_circulatory'] = one_hot_encode('Circulatory')
df_clean['diag1_endocrine'] = one_hot_encode('Endocrine')
df_clean['diag1_respiratory'] = one_hot_encode('Respiratory')
df_clean['diag1_digestive'] = one_hot_encode('Digestive')
df_clean['diag1_symptoms'] = one_hot_encode('Symptoms')
df_clean['diag1_injury_poisoning'] = one_hot_encode('Injury_Poisoning')
df_clean['diag1_genitourinary'] = one_hot_encode('Genitourinary')
df_clean['diag1_musculoskeletal'] = one_hot_encode('Musculoskeletal')
df_clean['diag1_neoplasms'] = one_hot_encode('Neoplasms')
df_clean['diag1_other'] = [1 if x not in diag_categories else 0 for x in data.category_diag_1]


In [ ]:
# create an additional column category_diag_1 to keep the string values of diag_1 
# which was OHE before to run in the algorithm 

diag_categories = ['Circulatory', 'Endocrine', 'Respiratory', 'Digestive', 
                   'Symptoms', 'Injury_Poisoning', 'Genitourinary', 'Musculoskeletal', 'Neoplasms']

data.loc[~data.category_diag_1.isin(diag_categories), 'category_diag_1'] = 'Other'

# insert this columns into df_clean
col = data['category_diag_1']
df_clean.insert(df_clean.columns.get_loc('number_inpatient') + 1, 'category_diag_1', col, allow_duplicates=True)


In [ ]:
print(df_clean.shape[1]-3, "features after cleaning the input data!\n")

print("Sample preview of Cleaned Data:\n")
print(df_clean.head(3))

In [ ]:
# df_clean.to_csv("clean_diabetic_data.csv", index = False)
# print("\nCleaned data frame is exported to the current directory\n")


### 6) Text Features


In [17]:
# use word count as features 

sample = ['problem of evil',
          'evil queen',
          'horizon problem']

In [18]:
from sklearn.feature_extraction.text import CountVectorizer

vec = CountVectorizer()
X = vec.fit_transform(sample)
X

<3x5 sparse matrix of type '<class 'numpy.int64'>'
	with 7 stored elements in Compressed Sparse Row format>

In [19]:
import pandas as pd
pd.DataFrame(X.toarray(), columns=vec.get_feature_names())

,evil,horizon,of,problem,queen
0,1,0,1,1,0
1,1,0,0,0,1
2,0,1,0,1,0


In [20]:
# term frequency-inverse document frequency (TF–IDF)

from sklearn.feature_extraction.text import TfidfVectorizer

vec = TfidfVectorizer()
X = vec.fit_transform(sample)
pd.DataFrame(X.toarray(), columns=vec.get_feature_names())

,evil,horizon,of,problem,queen
0,0.517856,0.000000,0.680919,0.517856,0.000000
1,0.605349,0.000000,0.000000,0.000000,0.795961
2,0.000000,0.795961,0.000000,0.605349,0.000000


## 3. Feature pipeline

https://jakevdp.github.io/PythonDataScienceHandbook/05.04-feature-engineering.html

In [ ]:
from sklearn.pipeline import make_pipeline

model = make_pipeline(Imputer(strategy='mean'),
                      PolynomialFeatures(degree=2),
                      LinearRegression())

In [ ]:
model.fit(X, y)  # X with missing values, from above
print(y)
print(model.predict(X))